In [7]:
import pandas as pd
import numpy as np
import ortools
from ortools.linear_solver import pywraplp
from ortools.graph import pywrapgraph

In [8]:
#Abrindo o arquivo e salvando seus dados, linha a linha
arquivo = open("instancias/instance7.txt", 'r')
texto = arquivo.readlines()
arquivo.close()
print(texto)

['11\n', '24\n', '1\n', '11\n', '1 2 20\n', '1 3 30\n', '1 4 20\n', '2 5 20\n', '2 7 5\n', '3 2 5 \n', '3 6 20\n', '4 3 5\n', '4 7 20\n', '5 8 20\n', '5 10 20\n', '6 2 5\n', '6 5 5\n', '6 7 5\n', '6 8 5\n', '6 9 20\n', '7 3 10\n', '7 9 20\n', '7 10 10\n', '8 9 6\n', '8 11 20\n', '9 11 30\n', '9 10 6\n', '10 11 25\n']


In [9]:
#Guardando dados nas variáveis
n_vertices = int(texto[0].split()[0])
n_arcos = int(texto[1].split()[0])
i_origem = int(texto[2].split()[0])
i_escoadouro = int(texto[3].split()[0])

pontoInicial = []
pontoFinal = []
capacidadeArco = []

for i in range(4, len(texto)):
    pontoInicial.append(int(texto[i].split()[0]))
    pontoFinal.append(int(texto[i].split()[1]))
    capacidadeArco.append(int(texto[i].split()[2]))

print(n_vertices, n_arcos, i_origem, i_escoadouro)
print(pontoInicial)
print(pontoFinal)
print(capacidadeArco)

11 24 1 11
[1, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 6, 6, 6, 7, 7, 7, 8, 8, 9, 9, 10]
[2, 3, 4, 5, 7, 2, 6, 3, 7, 8, 10, 2, 5, 7, 8, 9, 3, 9, 10, 9, 11, 11, 10, 11]
[20, 30, 20, 20, 5, 5, 20, 5, 20, 20, 20, 5, 5, 5, 5, 20, 10, 20, 10, 6, 20, 30, 6, 25]


In [10]:
#transformando o problema de fluxo máximo em fluxo de custo mínimo

#definindo todos os custos como zero
custo = []
for i in range(4, len(texto)):
    custo.append(0)

#calculando o fluxo máximo viável
soma = 0
for i in capacidadeArco:
    soma += i

F = int(soma) #Limite do fluxo máximo viável pela rede
M = 900 #Custo unitário máximo
U = 10000 #Capacidade máxima de arco

pontoInicial.append(int(i_origem))
pontoFinal.append(int(i_escoadouro))
capacidadeArco.append(int(U))
custo.append(int(M))

#definindo as demandas em cada nó
suprimentos = []
for i in range(int(n_vertices)+1):
    suprimentos.append(0)

suprimentos[i_origem] = F
suprimentos[i_escoadouro] = -F

print(soma)
print(pontoInicial)
print(pontoFinal)
print(capacidadeArco)
print(custo)
print(suprimentos)

352
[1, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 6, 6, 6, 7, 7, 7, 8, 8, 9, 9, 10, 1]
[2, 3, 4, 5, 7, 2, 6, 3, 7, 8, 10, 2, 5, 7, 8, 9, 3, 9, 10, 9, 11, 11, 10, 11, 11]
[20, 30, 20, 20, 5, 5, 20, 5, 20, 20, 20, 5, 5, 5, 5, 20, 10, 20, 10, 6, 20, 30, 6, 25, 10000]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 900]
[0, 352, 0, 0, 0, 0, 0, 0, 0, 0, 0, -352]


In [11]:
fluxo_custo_min = pywrapgraph.SimpleMinCostFlow()
fluxo_max = 0

for i in range(0, len(pontoInicial)):
    fluxo_custo_min.AddArcWithCapacityAndUnitCost(pontoInicial[i], pontoFinal[i], capacidadeArco[i], custo[i])

for i in range(0, len(suprimentos)):
    fluxo_custo_min.SetNodeSupply(i, suprimentos[i])

if fluxo_custo_min.Solve() == fluxo_custo_min.OPTIMAL:
    print('Custo Mínimo:', fluxo_custo_min.OptimalCost())
    print('')
    print('  Arco    Fluxo / Capacidade  Custo')
    for i in range(fluxo_custo_min.NumArcs()):
        custoArco = fluxo_custo_min.Flow(i) * fluxo_custo_min.UnitCost(i)
        print('%1s -> %1s   %3s  / %3s       %3s' % (fluxo_custo_min.Tail(i), fluxo_custo_min.Head(i), fluxo_custo_min.Flow(i), fluxo_custo_min.Capacity(i), custoArco))
        
        #soma fluxo máximo
        if fluxo_custo_min.Tail(i) == 1 and fluxo_custo_min.Capacity(i) < U:
            fluxo_max += fluxo_custo_min.Flow(i)
else:
    print('Não foi possível encontrar a solução.')

print("")
print("Fluxo Máximo = ", fluxo_max)

Custo Mínimo: 258300

  Arco    Fluxo / Capacidade  Custo
1 -> 2    20  /  20         0
1 -> 3    25  /  30         0
1 -> 4    20  /  20         0
2 -> 5    20  /  20         0
2 -> 7     5  /   5         0
3 -> 2     5  /   5         0
3 -> 6    20  /  20         0
4 -> 3     0  /   5         0
4 -> 7    20  /  20         0
5 -> 8     5  /  20         0
5 -> 10    20  /  20         0
6 -> 2     0  /   5         0
6 -> 5     5  /   5         0
6 -> 7     0  /   5         0
6 -> 8     5  /   5         0
6 -> 9    10  /  20         0
7 -> 3     0  /  10         0
7 -> 9    20  /  20         0
7 -> 10     5  /  10         0
8 -> 9     0  /   6         0
8 -> 11    10  /  20         0
9 -> 11    30  /  30         0
9 -> 10     0  /   6         0
10 -> 11    25  /  25         0
1 -> 11   287  / 10000       258300

Fluxo Máximo =  65
